In [6]:
# ensure you have necessary packages 
import transformers 
import datasets 
import torch

# Try OPT-125m (smallest)

In [7]:
from transformers import GPT2Tokenizer, OPTForCausalLM

model = OPTForCausalLM.from_pretrained("facebook/opt-125m")
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-125m")

prompt = "Hey, are you consciours? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
generated_sentence = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(generated_sentence)

Hey, are you consciours? Can you talk to me?
I'm not sure, but I'm sure I can talk to you


# Try Contriever 

First clone the contriever repo

```bash
git clone git@github.com:facebookresearch/contriever.git
```

In [ ]:
!git clone git@github.com:facebookresearch/contriever.git

In [4]:
import sys
# print(sys.path)
sys.path.append("./contriever")

from src.contriever import Contriever
from transformers import AutoTokenizer

model = Contriever.from_pretrained("facebook/contriever") 
tokenizer = AutoTokenizer.from_pretrained("facebook/contriever") #Load the associated tokenizer:


sentences = [
    "Where was Marie Curie born?",
    "Maria Sklodowska, later known as Marie Curie, was born on November 7, 1867.",
    "Born in Paris on 15 May 1859, Pierre Curie was the son of Eugène Curie, a doctor of French Catholic origin from Alsace."
]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings = model(**inputs)


score01 = embeddings[0] @ embeddings[1] #1.0473
score02 = embeddings[0] @ embeddings[2] #1.0095

print(score01)
print(score02)

Some weights of the model checkpoint at facebook/contriever were not used when initializing Contriever: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing Contriever from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Contriever from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor(1.0473, grad_fn=<DotBackward0>)
tensor(1.0095, grad_fn=<DotBackward0>)
